# Funções Auxiliares

## Métricas de desempenho tradicional

In [ ]:
def detectar_tipo_tarefa(y):
    import numpy as np
    y_array = np.array(y)

    if np.issubdtype(y_array.dtype, np.floating):
        return "regressao"

    n_classes = len(np.unique(y_array))
    if n_classes == 2:
        return "binaria"
    elif n_classes > 2:
        return "multiclasse"
    return "desconhecido"


In [ ]:
def desempenho_tradicional_binario(y_true, y_pred, explain=False):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    acc = accuracy_score(y_true, y_pred)
    print(f"Acurácia:  {acc:.4f}")
    if explain:
        print("🔹 Acurácia: proporção de acertos no total de exemplos. Boa se > 0.80, mas pode enganar em dados desbalanceados.")

    prec = precision_score(y_true, y_pred)
    print(f"Precisão:  {prec:.4f}")
    if explain:
        print("🔹 Precisão: entre os que foram preditos como positivos, quantos realmente são. Boa se você quer evitar falsos positivos.")

    rec = recall_score(y_true, y_pred)
    print(f"Recall:    {rec:.4f}")
    if explain:
        print("🔹 Recall: entre os positivos reais, quantos foram detectados. Boa se você quer evitar falsos negativos.")

    f1 = f1_score(y_true, y_pred)
    print(f"F1-Score:  {f1:.4f}")
    if explain:
        print("🔹 F1-Score: média harmônica entre precisão e recall. Equilibra ambos quando há desbalanceamento.")
    return acc, prec, rec, f1

In [ ]:
def desempenho_tradicional_multiclasse(y_true, y_pred, explain=False):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    acc = accuracy_score(y_true, y_pred)
    print(f"Acurácia:  {acc:.4f}")
    if explain:
        print("🔹 Acurácia: proporção de acertos entre todas as classes. Boa se > 0.80, mas cuidado com desbalanceamento.")

    prec = precision_score(y_true, y_pred, average='macro')
    print(f"Precisão (macro): {prec:.4f}")
    if explain:
        print("🔹 Precisão: média da precisão de cada classe. Mostra se o modelo é justo com todas as classes.")

    rec = recall_score(y_true, y_pred, average='macro')
    print(f"Recall (macro):   {rec:.4f}")
    if explain:
        print("🔹 Recall: média do recall de cada classe. Mede cobertura média de cada classe verdadeira.")

    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"F1-Score (macro): {f1:.4f}")
    if explain:
        print("🔹 F1-Score: média harmônica entre precisão e recall para todas as classes.")
    return acc, prec, rec, f1

In [ ]:
def desempenho_tradicional_regressao(y_true, y_pred, explain=False):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    r2 = r2_score(y_true, y_pred)
    print(f"R2:  {r2:.4f}")
    if explain:
        print("🔹 R2: quão bem o modelo explica a variância. 1.0 é perfeito. Pode ser negativo se o modelo for ruim.")

    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {mse:.4f}")
    if explain:
        print("🔹 MSE: erro quadrático médio. Penaliza mais os erros grandes. Quanto menor, melhor.")

    mae = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {mae:.4f}")
    if explain:
        print("🔹 MAE: erro absoluto médio. Indica o erro médio em termos absolutos. Quanto menor, melhor.")
    return r2, mse, mae

In [5]:
def desempenho_tradicional(model, X_test, y_test, explain=False):
    tipo = detectar_tipo_tarefa(y_test)

    y_pred = model.predict(X_test)

    if tipo == "binaria":
        return desempenho_tradicional_binario(y_test, y_pred, explain)
    elif tipo == "multiclasse":
        return desempenho_tradicional_multiclasse(y_test, y_pred, explain)
    elif tipo == "regressao":
        return desempenho_tradicional_regressao(y_test, y_pred, explain)
    else:
        print("⚠️ Tipo de tarefa não reconhecido.")
        return None


## Sumário

In [5]:
def summarize_dataset(df):
    """
    📊 Gera um sumário completo de estatísticas e estrutura do dataset.

    Parâmetros:
    - df: pandas DataFrame

    Retorna:
    - Imprime métricas descritivas e insights úteis sobre as colunas
    """
    import pandas as pd
    import numpy as np

    print("📦 Tamanho do dataset:")
    print(f"- Linhas: {df.shape[0]}")
    print(f"- Colunas: {df.shape[1]}")
    print("\n📄 Primeiras 5 linhas:")
    display(df.head())

    print("\n🔢 Informações básicas:")
    display(df.info())

    print("\n📈 Estatísticas descritivas:")
    display(df.describe().T)

    print("\n🧾 Colunas categóricas/dummies:")
    cat_cols = [col for col in df.columns if df[col].nunique() <= 10 and df[col].dtype in [np.int64, np.bool_, np.int32, object]]
    for col in cat_cols:
        print(f"- {col}: {df[col].unique()} (n={df[col].nunique()})")

    print("\n📊 Correlação entre variáveis numéricas:")
    corr = df.corr(numeric_only=True)
    display(corr.style.background_gradient(cmap='coolwarm', axis=None).format(precision=2))


    print("\n⚠️ Colunas com valores nulos:")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        display(missing)
    else:
        print("✅ Nenhum valor nulo encontrado.")


In [6]:
def dist_racial(df,race_columns):
    from IPython.display import display, HTML

    # 📊 Loop para exibir totais e distribuições lado a lado
    for race in race_columns:
        print(f"\n📊 Raça: {race}")
        total = df[race].sum()
        print(f"Total de indivíduos: {int(total)}")

        # 📊 Gerar tabelas
        dist_abs = pd.crosstab(df[race], df['Reincidente'])
        dist_pct = pd.crosstab(df[race], df['Reincidente'], normalize='index').round(3)

        # 🔗 Combinar HTML das duas tabelas
        html = f"""
        <div style="display: flex; gap: 40px;">
            <div>
                <b>Distribuição absoluta:</b>
                {dist_abs.to_html()}
            </div>
            <div>
                <b>Distribuição percentual:</b>
                {dist_pct.to_html()}
            </div>
        </div>
        """
        display(HTML(html))


## Métricas de Disparidade/Fairness

In [ ]:

def consistency_score(X, y_pred, k=5):
    from sklearn.metrics import pairwise_distances
    from sklearn.neighbors import NearestNeighbors
    """
    📏 Mede Individual Fairness com base na Consistência.
    Quanto maior, mais justo o modelo.
    
    Parâmetros:
    - X: Features (sem atributo sensível)
    - y_pred: Saídas do modelo (classes ou probabilidades)
    - k: Número de vizinhos

    Retorna:
    - Média da similaridade de predições entre vizinhos
    """
    nn = NearestNeighbors(n_neighbors=k+1)  # +1 porque inclui o próprio ponto
    nn.fit(X)
    neighbors = nn.kneighbors(X, return_distance=False)[:, 1:]  # Remove o próprio índice

    diffs = []
    for i in range(X.shape[0]):
        neighbor_preds = y_pred[neighbors[i]]
        diffs.append(np.mean(np.abs(y_pred[i] - neighbor_preds)))

    return 1 - np.mean(diffs)


## Explicabilidade

In [ ]:
def explain_model(model, X, max_display=10, nome=None, target_name=None):
    import shap
    import matplotlib.pyplot as plt
    from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
    from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
    """
    Explica modelos de árvore com SHAP (suporta classificadores e regressões).
    
    Parâmetros:
        - model: modelo treinado (ex: DecisionTree, GBC, GBR)
        - X: dados de entrada (DataFrame ou array)
        - max_display: número máximo de features no gráfico
        - nome: título opcional a ser incluído nos gráficos
    """
    
    modelos_problema = isinstance(model, DecisionTreeClassifier)
    modelos_sem_problema = isinstance(model, (
        GradientBoostingClassifier,
        DecisionTreeRegressor,
        GradientBoostingRegressor
    ))

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    (f"shap_values.shape = {shap_values.shape}, X.shape = {X.shape}")
    
    if not modelos_problema:
        if not modelos_sem_problema:
            print("⚠️ Tipo de modelo não identificado, tentando procedimento padrão")
            if shap_values.shape != X.shape:
                raise AssertionError(f"shap_values.shape = {shap_values.shape}, X.shape = {X.shape}")
                return
        
        print("🔍 Gerando explicabilidade global do modelo...")

        

        shap.summary_plot(shap_values, X, plot_type="bar", max_display=max_display, show=False)
        if nome:
            plt.title(f"{nome} — Importância global")
        plt.show()

        shap.summary_plot(shap_values, X, max_display=max_display, show=False)
        if nome:
            plt.title(f"{nome} — Distribuição dos impactos")
        plt.show()
    
    elif isinstance(model, DecisionTreeClassifier):
        print(f"shap_values.shape = {shap_values.shape}, X.shape = {X.shape}")
        class_names = model.classes_

        _, _, n_classes = shap_values.shape
        for i in range(n_classes):
            
            print(f"\n📊 {target_name}: {i}' — Explicação global")
            shap.summary_plot(shap_values[:, :, i], X, plot_type="bar", max_display=max_display, show=False)
            if nome:
                plt.title(f"{nome} — {target_name}: {i}' — Importância global")
            plt.show()

            print(f"📈 {target_name}: {i}' — Distribuição dos impactos")
            shap.summary_plot(shap_values[:, :, i], X, max_display=max_display, show=False)
            if nome:
                plt.title(f"{nome} — {target_name}: {i}' — Distribuição dos impactos")
            plt.show()


In [ ]:
def explain_individual(index, model, X_train):
    import shap
    from scipy.special import expit
    import matplotlib.pyplot as plt
    """
    🧠 Explicabilidade local com SHAP (versão waterfall).
    Mostra o impacto de cada feature + exibe a probabilidade prevista no gráfico.
    """
    # Obter valores SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)
    instance = X_train.iloc[[index]]

    # Corrigir acesso ao escalar
    expected_value = explainer.expected_value[0]
    fx = float(shap_values[index].sum() + expected_value)
    prob = float(expit(fx))

    # Construir explicação
    explanation = shap.Explanation(
        values=shap_values[index],
        base_values=expected_value,
        data=instance.values[0],
        feature_names=instance.columns
    )

    # Plot com título seguro (sem emoji)
    shap.plots.waterfall(explanation, show=False)
    plt.title(f"SHAP Waterfall — Instância {index} | Prob. prevista: {prob:.2f}", fontsize=12)
    plt.show()


## 🚀 Pipeline Models e Mitigações

In [ ]:
def pipeline_baseline(X_train, X_test, y_train, y_test, weights=None, label="Baseline"):
    from sklearn.metrics import classification_report
    from sklearn.ensemble import GradientBoostingClassifier
    """
    Pipeline simples de treinamento e avaliação com GradientBoostingClassifier.
    """
    model = GradientBoostingClassifier(random_state=42)
    model.fit(X_train, y_train, sample_weight=weights)
    y_pred = model.predict(X_test)
    
    print(f"\n🔍 Resultados para: {label}")
    print(classification_report(y_test, y_pred))
    return model

In [ ]:
def pipeline_fairshap(X_train, X_test, y_train, y_test,
                       protected_attribute_col,
                       privileged_value, unprivileged_value,
                       label_favorable, label_unfavorable
                       ):
    from fairSV.fair_shapley import FairShapley
    """
    Pipeline com FairShap aplicado, incluindo:
    - Cálculo da matriz SV.
    - Cálculo dos Shapley Values fairness-aware.
    - Geração de pesos baseados em uma métrica de justiça.

    Parâmetros:
    - X_train, X_test: Dados de treino e teste (DataFrame ou array).
    - y_train, y_test: Labels.
    - protected_attribute_col: Nome da coluna do atributo sensível.
    - privileged_value: Valor do grupo privilegiado (ex.: 1).
    - unprivileged_value: Valor do grupo não privilegiado (ex.: 0).
    - label_favorable: Label favorável (ex.: 0).
    - label_unfavorable: Label desfavorável (ex.: 1).

    Retorna:
    - weights: Vetor de pesos gerados a partir de Equal Opportunity Normalizado.
    - fair_sv_extractor: Objeto FairShapley com todos os resultados e métricas.
    """
    # ✅ Extrair os valores do atributo sensível no conjunto de teste
    protected_values = X_test[protected_attribute_col].values

    # ✔️ Definir o dicionário do FairShap
    protected_attributes_dict = {
        'values': protected_values,
        'privileged_protected_attribute': privileged_value,
        'unprivileged_protected_attribute': unprivileged_value,
        'favorable_label': label_favorable,
        'unfavorable_label': label_unfavorable
    }

    # 🚀 Criar o objeto FairShap
    fair_sv_extractor = FairShapley(
        X_train.values, y_train,
        X_test.values, y_test,
        protected_attributes_dict=protected_attributes_dict,
        show_plot=False,
        calculate_2dim=True
    )

    # 🔍 Encontrar o melhor K
    best_k, _, _ = fair_sv_extractor.get_best_K()

    # 🔢 Calcular a matriz SV
    _ = fair_sv_extractor.get_SV_matrix(K=best_k)

    # 🔢 Calcular os Shapley Values fairness-aware
    fair_sv_extractor.get_sv_arrays()

    # 🎯 Geração dos pesos com base na Equal Opportunity
    sv_eop = fair_sv_extractor.sv_equal_opportunity_difference
    #normalização
    weights = (sv_eop - np.min(sv_eop)) / (np.max(sv_eop) - np.min(sv_eop))

    return weights, fair_sv_extractor
